# Day 24: Never Tell Me The Odds
It seems like something is going wrong with the snow-making process. Instead of forming snow, the water that's been absorbed into the air seems to be forming hail!

Maybe there's something you can do to break up the hailstones?

Due to strong, probably-magical winds, the hailstones are all flying through the air in perfectly linear trajectories. You make a note of each hailstone's position and velocity (your puzzle input). For example:
```
19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3
```
Each line of text corresponds to the position and velocity of a single hailstone. The positions indicate where the hailstones are right now (at time 0). The velocities are constant and indicate exactly how far each hailstone will move in one nanosecond.

Each line of text uses the format px py pz @ vx vy vz. For instance, the hailstone specified by 20, 19, 15 @ 1, -5, -3 has initial X position 20, Y position 19, Z position 15, X velocity 1, Y velocity -5, and Z velocity -3. After one nanosecond, the hailstone would be at 21, 14, 12.

Perhaps you won't have to do anything. How likely are the hailstones to collide with each other and smash into tiny ice crystals?

To estimate this, consider only the X and Y axes; ignore the Z axis. Looking forward in time, how many of the hailstones' paths will intersect within a test area? (The hailstones themselves don't have to collide, just test for intersections between the paths they will trace.)

In this example, look for intersections that happen with an X and Y position each at least 7 and at most 27; in your actual data, you'll need to check a much larger test area. Comparing all pairs of hailstones' future paths produces the following results:
```
Hailstone A: 19, 13, 30 @ -2, 1, -2
Hailstone B: 18, 19, 22 @ -1, -1, -2
Hailstones' paths will cross inside the test area (at x=14.333, y=15.333).

Hailstone A: 19, 13, 30 @ -2, 1, -2
Hailstone B: 20, 25, 34 @ -2, -2, -4
Hailstones' paths will cross inside the test area (at x=11.667, y=16.667).

Hailstone A: 19, 13, 30 @ -2, 1, -2
Hailstone B: 12, 31, 28 @ -1, -2, -1
Hailstones' paths will cross outside the test area (at x=6.2, y=19.4).

Hailstone A: 19, 13, 30 @ -2, 1, -2
Hailstone B: 20, 19, 15 @ 1, -5, -3
Hailstones' paths crossed in the past for hailstone A.

Hailstone A: 18, 19, 22 @ -1, -1, -2
Hailstone B: 20, 25, 34 @ -2, -2, -4
Hailstones' paths are parallel; they never intersect.

Hailstone A: 18, 19, 22 @ -1, -1, -2
Hailstone B: 12, 31, 28 @ -1, -2, -1
Hailstones' paths will cross outside the test area (at x=-6, y=-5).

Hailstone A: 18, 19, 22 @ -1, -1, -2
Hailstone B: 20, 19, 15 @ 1, -5, -3
Hailstones' paths crossed in the past for both hailstones.

Hailstone A: 20, 25, 34 @ -2, -2, -4
Hailstone B: 12, 31, 28 @ -1, -2, -1
Hailstones' paths will cross outside the test area (at x=-2, y=3).

Hailstone A: 20, 25, 34 @ -2, -2, -4
Hailstone B: 20, 19, 15 @ 1, -5, -3
Hailstones' paths crossed in the past for hailstone B.

Hailstone A: 12, 31, 28 @ -1, -2, -1
Hailstone B: 20, 19, 15 @ 1, -5, -3
Hailstones' paths crossed in the past for both hailstones.
```
So, in this example, 2 hailstones' future paths cross inside the boundaries of the test area.

However, you'll need to search a much larger test area if you want to see if any hailstones might collide. Look for intersections that happen with an X and Y position each at least 200000000000000 and at most 400000000000000. Disregard the Z axis entirely.

Considering only the X and Y axes, check all pairs of hailstones' future paths for intersections. How many of these intersections occur within the test area?

In [65]:
test_area = ((2e14, 2e14), (4e14, 4e14))
test_area_min, test_area_max = 200000000000000, 400000000000000

In [1]:
paths = []
with open(r"advent_of_code_2023\24.txt", "r") as file:
    while l := file.readline():
        path = l.rstrip("\n")
        pos, vel = path.split("@")[0].split(","), path.split("@")[1].split(",")
        vec = pos + vel
        vec = list(map(int, vec))
        paths.append(vec)

In [48]:
import numpy as np
from scipy.linalg import pinv

def coll_pos(a, b):
    """
    x1 = a[0] + a[3]*t
    y1 = a[1] + a[4]*t
    z1 = a[2] + a[5]*t
    
    x2 = b[0] + b[3]*s
    y2 = b[1] + b[4]*s
    z2 = b[2] + b[5]*s
    
    a[3]*t - b[3]*s = b[0]-a[0]
    a[4]*t - b[4]*s = b[1]-a[1]
    a[5]*t - b[5]*s = b[2]-a[2]
    """
    A = np.array([[a[3], - b[3]],
                  [a[4], - b[4]],
                  [a[5], - b[5]]])
    B = np.array([b[0]-a[0],
                  b[1]-a[1],
                  b[2]-a[2]])
    
    A_ = pinv(A)
    
    # Verify it exist one solution:
    res = np.einsum("ij,jk,k -> k", A, A_, B)
    if not np.allclose(res, B):
        return None, None
    
    sol = np.einsum("ij,j -> i", A_, B)
    x, y, z = a[0] + a[3]*sol[0], a[1] + a[4]*sol[0], a[2] + a[5]*sol[0]
    return (x, y, z), sol

In [68]:
tot = 0
for idx, path in enumerate(paths):
    for path_2 in paths[idx+1:]:
        path_ = path.copy()
        path_2_ = path_2.copy()
        path_[2], path_[5] = 0, 0
        path_2_[2], path_2_[5] = 0, 0
        col_pos, times = coll_pos(path_, path_2_)
        if col_pos is None:
            print(f"These Lines did not intersect: {path, path_2}")
        elif  test_area_min<col_pos[0]<test_area_max and test_area_min<col_pos[1]<test_area_max \
            and all(t>0 for t in times):
            tot += 1

[-1.10757978e+13 -2.92529787e+14  0.00000000e+00] [ -58543502553908 -257704812535484                0]
These Lines did not intersect: ([443280332788269, 407337432855918, 469184960570809, -32, -192, -175], [384736830234361, 149632620320434, 273738290851961, 13, 78, 20])
[-4.44848734e+13 -3.91602304e+14  0.00000000e+00] [-103260486916237 -325328686822930                0]
These Lines did not intersect: ([415277507456164, 404222039954908, 391326191866208, -68, -190, -123], [312017020539927, 78893353131978, 33381762671555, 68, 190, 301])
[ 1.19894012e+14 -6.14872887e+13  0.00000000e+00] [111439946814584 -67542855032725               0]
These Lines did not intersect: ([140621510449133, 378286508510411, 275757933634227, 143, 121, 42], [252061457263717, 310743653477686, 245756821390385, 13, 11, 86])
[6.69192904e+12 2.06813218e+14 0.00000000e+00] [ 21220459197070 174884159507820               0]
These Lines did not intersect: ([243684834855157, 152707452857582, 374779262821569, 72, 270, -163],

In [49]:
tot = 0
for idx, path in enumerate(paths):
    for path_2 in paths[idx+1:]:
        col_pos, times = coll_pos(path, path_2)
        if col_pos is None:
            continue
            # print(f"These Lines did not intersect: {path, path_2}")
        print(f"Col Pos {col_pos}, at time: {times}")
        # elif  test_area_min<col_pos[0]<test_area_max and test_area_min<col_pos[1]<test_area_max \
        #     and all(t>0 for t in times):
        #     tot += 1

# Part Two
Upon further analysis, it doesn't seem like any hailstones will naturally collide. It's up to you to fix that!

You find a rock on the ground nearby. While it seems extremely unlikely, if you throw it just right, you should be able to hit every hailstone in a single throw!

You can use the probably-magical winds to reach any integer position you like and to propel the rock at any integer velocity. Now including the Z axis in your calculations, if you throw the rock at time 0, where do you need to be so that the rock perfectly collides with every hailstone? Due to probably-magical inertia, the rock won't slow down or change direction when it collides with a hailstone.

In the example above, you can achieve this by moving to position 24, 13, 10 and throwing the rock at velocity -3, 1, 2. If you do this, you will hit every hailstone as follows:
```
Hailstone: 19, 13, 30 @ -2, 1, -2
Collision time: 5
Collision position: 9, 18, 20

Hailstone: 18, 19, 22 @ -1, -1, -2
Collision time: 3
Collision position: 15, 16, 16

Hailstone: 20, 25, 34 @ -2, -2, -4
Collision time: 4
Collision position: 12, 17, 18

Hailstone: 12, 31, 28 @ -1, -2, -1
Collision time: 6
Collision position: 6, 19, 22

Hailstone: 20, 19, 15 @ 1, -5, -3
Collision time: 1
Collision position: 21, 14, 12
```
Above, each hailstone is identified by its initial position and its velocity. Then, the time and position of that hailstone's collision with your rock are given.

After 1 nanosecond, the rock has exactly the same position as one of the hailstones, obliterating it into ice dust! Another hailstone is smashed to bits two nanoseconds after that. After a total of 6 nanoseconds, all of the hailstones have been destroyed.

So, at time 0, the rock needs to be at X position 24, Y position 13, and Z position 10. Adding these three coordinates together produces 47. (Don't add any coordinates from the rock's velocity.)

Determine the exact position and velocity the rock needs to have at time 0 so that it perfectly collides with every hailstone. What do you get if you add up the X, Y, and Z coordinates of that initial position?

In [43]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np

ax = plt.figure().add_subplot(projection='3d')

# Make the grid
x, y, z, u, v, w = [], [], [], [], [], []
for idx, path in enumerate(paths[:]):
    x.append(path[0])
    y.append(path[1])
    z.append(path[2])
    u.append(path[3])
    v.append(path[4])
    w.append(path[5])
ax.quiver(x, y, z, u, v, w, length=0.5e14, normalize=True)

plt.show()

In [68]:
from scipy.optimize import root, least_squares

neq = 30
def func(x):
    res = []
    for i, path in enumerate(paths[:neq]):
        res.append(x[0] + x[3]*x[i+6] - path[0] - path[3]*x[i+6])
        res.append(x[1] + x[4]*x[i+6] - path[1] - path[4]*x[i+6])
        res.append(x[2] + x[5]*x[i+6] - path[2] - path[5]*x[i+6])
    return res

sol = least_squares(func, [1e12]*(6+neq), method="lm")
print(sol)
print(sol.x)
print(sum(sol.x[:3]))

     message: `xtol` termination condition is satisfied.
     success: True
      status: 3
         fun: [ 3.906e-03  1.562e-02 ...  0.000e+00  0.000e+00]
           x: [ 1.312e+14  3.993e+14 ...  7.078e+11  4.550e+11]
        cost: 0.0037184059619903564
         jac: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
               [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
               ...
               [ 0.000e+00  1.000e+00 ...  0.000e+00 -3.780e+02]
               [ 0.000e+00  0.000e+00 ...  0.000e+00 -1.390e+02]]
        grad: [-2.661e-02  1.270e-02 ...  0.000e+00  0.000e+00]
  optimality: 23049188866.32584
 active_mask: [0 0 ... 0 0]
        nfev: 892
        njev: None
[ 1.31246724e+14  3.99310845e+14  2.77550172e+14  2.79000000e+02
 -1.84000000e+02  1.60000000e+01  1.30621773e+11  4.23178591e+11
  6.31793974e+11  5.52253912e+11  5.74469437e+11  5.32225741e+11
  3.30627052e+11  4.20587698e+11  1.00332350e+12  7.27555405e+11
  8.51878471e+11  5.47958426e+11  6.2494508